When u run in colab, make colab flag True.

In [1]:
colab=False #True/False: #When u run in colab, make this flag True.

In [2]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive')

In [3]:
if colab:
    #Add a shortcut of project folder to your drive's home for this to work
    !ls '/content/drive/MyDrive/Traffic'
    root = '/content/drive/MyDrive/Traffic'
else:
    import os
    root = os.getcwd()
root

'/home/skycam/sample_projs/FRCNN/delivered/FastRCNN_Traffic'

In [4]:
#!unzip "/content/drive/MyDrive/Traffic/Traffic_FRCNN.zip" -d "/content/drive/MyDrive/Traffic/"

Uncomment above line after putting zip file inside Traffic folder. Create another notebook and put above cells in it do this unzipping

In [5]:
if colab:
    import sys
    sys.path.append(root)

In [6]:
if colab:
    !pip3 install -r '/content/drive/MyDrive/Traffic/requirements.txt'
else:
    !pip3 install -r requirements.txt

In [7]:
import os
from utilities.data_utils.Dataset import DatasetGen, get_transform
from utilities.utils import collate_fn
from utilities.train_eval.engine import train_one_epoch, evaluate, get_model_result
import glob

import nvidia_smi # for python 3, you need nvidia-ml-py3 library

import torch
torch.cuda.empty_cache()
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead

In [8]:
PATH = 'Weight.pth'

output_image_folder = 'output'

proj = 'fabric' # 'sign','fabric' (put ur project name here)

if proj == 'sign':
    classes = ['warning', 'prohibitory', 'mandatory']
elif proj == 'fabric':
    classes = ['Hole', 'Line', 'Stain']
else:
    classes = ['bg']

num_classes = len(classes)+1  # n class + background

batch_size = 3

num_epochs = 5

In [9]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

Total memory: 8505131008
Free memory: 8175026176
Used memory: 330104832


In [10]:
dataset_train = DatasetGen(root+'/data/train', get_transform(horizontal_flip=True),classes=classes)
dataset_test = DatasetGen(root+'/data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_train = torch.utils.data.DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0,
        collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
device

device(type='cuda')

In [12]:
anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                        aspect_ratios=([1.0, 1.0, 1.0, 1.0], 
                                                     [0.8, 1.0, 1.0, 1.0], 
                                                     [1.0, 0.8, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0]))

In [13]:
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [14]:
if os.path.isfile(PATH):
    model.load_state_dict(torch.load(PATH))

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

## Training
Change 'if 0:' to 'if 1:' for training

In [21]:
if 0:#make this 1 for training to happen
    for epoch in range(num_epochs):
        res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
        print(f'gpu: {res.gpu}%, gpu-mem: {res.memory}%')

        train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=1)
        lr_scheduler.step()
        evaluate(model, data_loader_test, device=device)

    print("Training complete!")
    
    torch.cuda.synchronize()
    # create directory for saving the model
    print("Saving model...")
    torch.save(model.state_dict(), PATH)
    print("Model saving complete!")
    nvidia_smi.nvmlShutdown()

gpu: 1%, gpu-mem: 0%
Epoch: [0]  [ 0/13]  eta: 0:00:15  lr: 0.000421  loss: 0.3781 (0.3781)  loss_classifier: 0.0172 (0.0172)  loss_box_reg: 0.0108 (0.0108)  loss_objectness: 0.0365 (0.0365)  loss_rpn_box_reg: 0.3136 (0.3136)  time: 1.2239  data: 0.3048  max mem: 3898
Epoch: [0]  [ 1/13]  eta: 0:00:13  lr: 0.000837  loss: 0.3443 (0.3612)  loss_classifier: 0.0172 (0.0234)  loss_box_reg: 0.0108 (0.0157)  loss_objectness: 0.0365 (0.0366)  loss_rpn_box_reg: 0.2575 (0.2855)  time: 1.0875  data: 0.3107  max mem: 3898
Epoch: [0]  [ 2/13]  eta: 0:00:11  lr: 0.001254  loss: 0.3443 (0.3333)  loss_classifier: 0.0246 (0.0238)  loss_box_reg: 0.0151 (0.0155)  loss_objectness: 0.0365 (0.0339)  loss_rpn_box_reg: 0.2575 (0.2602)  time: 1.0563  data: 0.2742  max mem: 4012
Epoch: [0]  [ 3/13]  eta: 0:00:10  lr: 0.001670  loss: 0.2776 (0.3175)  loss_classifier: 0.0238 (0.0238)  loss_box_reg: 0.0151 (0.0164)  loss_objectness: 0.0284 (0.0311)  loss_rpn_box_reg: 0.2094 (0.2461)  time: 1.0296  data: 0.2719  m

KeyboardInterrupt: 

In [16]:
# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)

## Evaluate

In [17]:
evaluate(model, data_loader_test, device=device)

n_threads =  4
creating index...
index created!
Test:  [0/3]  eta: 0:00:00  model_time: 0.1073 (0.1073)  evaluator_time: 0.0006 (0.0006)  time: 0.1358  data: 0.0256  max mem: 523
Test:  [2/3]  eta: 0:00:00  model_time: 0.1107 (0.1258)  evaluator_time: 0.0005 (0.0005)  time: 0.1588  data: 0.0306  max mem: 588
Test: Total time: 0:00:00 (0.1591 s / it)
Averaged stats: model_time: 0.1107 (0.1258)  evaluator_time: 0.0005 (0.0005)
Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (A

## Inference 
Repeating all the process to ensure nothing is taken from training stage
Check output folder for results

In [18]:
%load_ext autoreload
%autoreload 2

In [19]:
%reload_ext autoreload
%autoreload 2

In [20]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

dataset_test = DatasetGen(root+'/data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                   aspect_ratios=([1.0, 1.0, 1.0, 1.0],
                                                  [0.8, 1.0, 1.0, 1.0],
                                                  [1.0, 0.8, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0]))
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load(PATH))
model.to(device)

# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)
# write testing result to output folder
for img_idx, batch_sampler in enumerate(data_loader_test):
    img_test = batch_sampler[0][0]
    target_test = batch_sampler[1][0]
    i = target_test["image_id"].item()
    print('\nimage id',i)
    get_model_result(img_test, model, target_test, i, device, location=output_image_folder, threshold=0.2,classes=classes)

print("Testing complete!")

torch.cuda.synchronize()
nvidia_smi.nvmlShutdown()

Total memory: 8505131008
Free memory: 7438598144
Used memory: 1066532864

image id 0
initial boxes: ['warning', 'mandatory', 'prohibitory']
True boxes: mandatory tensor([727., 265., 831., 420.])
Final pred boxes: warning [698.4111328125, 282.22772216796875, 885.4801635742188, 439.16778564453125]

image id 1
initial boxes: ['mandatory', 'warning', 'mandatory', 'warning', 'prohibitory', 'mandatory', 'warning', 'prohibitory', 'prohibitory', 'mandatory', 'prohibitory']
True boxes: mandatory tensor([1162.,  126., 1225.,  177.])
Final pred boxes: mandatory [1151.7557373046875, 110.57440948486328, 1225.782958984375, 174.7506561279297]
Final pred boxes: mandatory [1168.392822265625, 131.7520751953125, 1216.6466064453125, 171.8172149658203]

image id 2
initial boxes: ['mandatory', 'mandatory', 'prohibitory', 'warning', 'mandatory', 'prohibitory', 'warning', 'mandatory', 'warning', 'mandatory', 'mandatory', 'prohibitory', 'mandatory', 'mandatory', 'prohibitory', 'warning', 'mandatory', 'warning'

Check output folder to see the result images